# difflib

[doc](https://docs.python.org/3/library/difflib.html#module-difflib)

In [1]:
import difflib

s1, s2 = 'Miloš Zeman', 'Malej Eman'
difflib.SequenceMatcher(None, s1, s2).ratio() #maybe you don't need levensthein, jaro-winkler et al.

0.5714285714285714

In [2]:
difflib.SequenceMatcher(None, s1, s2).find_longest_match(0,len(s1), 0, len(s2))

Match(a=8, b=7, size=3)

In [3]:
for m in difflib.SequenceMatcher(lambda x:x==' ', s1, s2).get_matching_blocks():
    print(s1[m.a:m.a+m.size], s2[m.b:m.b+m.size])


M M
l l
e e
man man
 


# tempfile

In [4]:
import tempfile, shutil, os

#temp "file-like obj"
with tempfile.TemporaryFile('w+t') as tmp:
    tmp.write('pyvo!')
    print(tmp.name)

58


In [5]:
#temp file
with tempfile.NamedTemporaryFile() as tmp2:
    tmp2.write(b'dve piva')
    print(tmp2.name)
    
print(os.path.exists(tmp2.name))

/tmp/tmpqzdphqyw
False


In [6]:
#temp dir
tmp_dir = tempfile.mkdtemp()
print(tmp_dir)
shutil.rmtree(tmp_dir) # :(

/tmp/tmpxb6voltw


In [7]:
#temp auto destruct dir
with tempfile.TemporaryDirectory() as tmp_dir2:
    f1, f2 = tempfile.NamedTemporaryFile(dir=tmp_dir2), tempfile.NamedTemporaryFile(dir=tmp_dir2)
    f1.write(b'mrkev')
    f1.write(b'klobasa')
    print(os.listdir(tmp_dir2))
print(os.path.exists(tmp_dir2))

['tmpgrj3ivk_', 'tmpv7mxcvlx']
False


# inspect

"The inspect module provides several useful functions to help get information about live objects such as modules, classes, methods, functions, tracebacks, frame objects, and code objects."

[doc](https://docs.python.org/3/library/inspect.html)

my usage: 
 - poor man's "production debugger"
 - expose particular functions as cmd choices

In [8]:
import inspect

def say_ni(times=3):
    print('NI!'%times)

In [9]:
inspect.getsource(say_ni)

"def say_ni(times=3):\n    print('NI!'%times)\n"

In [10]:
inspect.getfile(say_ni)

'<ipython-input-8-2f00b2bf0675>'

In [11]:
inspect.signature(say_ni).parameters

mappingproxy({'times': <Parameter "times=3">})

In [12]:
import os

inspect.getmembers(os, inspect.ismodule)

[('abc', <module 'abc' from '/usr/lib/python3.6/abc.py'>),
 ('errno', <module 'errno' (built-in)>),
 ('path', <module 'posixpath' from '/usr/lib/python3.6/posixpath.py'>),
 ('st', <module 'stat' from '/usr/lib/python3.6/stat.py'>),
 ('sys', <module 'sys' (built-in)>)]

In [13]:
inspect.getmembers(os, lambda x: inspect.ismodule(x) and x.__name__.startswith('s'))

[('st', <module 'stat' from '/usr/lib/python3.6/stat.py'>),
 ('sys', <module 'sys' (built-in)>)]

# operator

- attrgetter, itemgetter, methodcaller
- operators :)

In [14]:
import operator, collections, random

t = collections.namedtuple('A', 'b')

list(map(operator.attrgetter('b'), [t(b=1)]*20)) #list(map(lambda x: getattr(x, 'b'), [t(b=1)]*20))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [15]:
list(map(operator.itemgetter(5), [range(10)]*20)) #list(map(lambda x: x[5], [t(b=1)]*20))

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [16]:
def my_filter(items, predicate): 
    return filter(predicate, items)

list(my_filter(['abcd', 'cdef', 'zaza'], operator.methodcaller('startswith', 'a'))) # getattr()(*args)

['abcd']

In [17]:
import operator, re

def check_ver(predicates, version):
    OP_MAP = {'==': operator.eq, '<': operator.lt, '>': operator.gt, '<=': operator.le, '>=': operator.ge}
    RE_PREDICATE = r'(?P<what>.*)\s?(?P<op>==|<|>|<=|>={1,2})\s?(?P<version>[\d\.]+);?'
    rules = [m.groupdict() for m in re.finditer(RE_PREDICATE, predicates)]
    
    return all([OP_MAP[r['op']](version, float(r['version'])) for r in rules])

print(check_ver('x==2.7', 2.7))
print(check_ver('x>=2;x<3', 3.6))
print(check_ver('x>=2;x<3', 2.7))

True
False
True


# webbrowser

basically spawns a browser with given url (also local file)

In [18]:
import webbrowser, io

html = io.open('/tmp/meh.html', 'w+t')
html.write('<marquee><h1>Batteries included</h1></marquee>')
html.close()
webbrowser.open(html.name)

True

# audioop, wave

In [19]:
import audioop, wave, io, webbrowser, tempfile

berry = './berries.wav' #http://www.dailywav.com/quotes/monty-python-and-holy-grail/your-father-smelt-elderberries
webbrowser.open(os.path.abspath(berry))

True

In [20]:
af = wave.open(berry)
print('channels: ', af.getnchannels())
print('compressed?: ', af.getcompname())
print('rate [Hz]: ', af.getframerate())

channels:  1
compressed?:  not compressed
rate [Hz]:  11025


In [21]:
af = wave.open(berry)
out = io.open('/tmp/raf.wav', 'wb')
raf = wave.open(out)
raf.setparams((af.getnchannels(), af.getsampwidth(), af.getframerate(), '', af.getcomptype(), af.getcompname()))
rev = audioop.reverse(af.readframes(af.getnframes()), af.getnchannels())
raf.writeframesraw(rev)
webbrowser.open(out.name)

True